In [12]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset
import biosppy.signals.ecg as ecg
import matplotlib.pyplot as plt
import neurokit2 as nk
import heartpy as hp
from statistics import mean
import functions
from multiprocessing import Pool
from datetime import time
import functions
import scipy
import math
import operator
from functions import *
import itertools
from sklearn.ensemble import ExtraTreesClassifier


In [39]:
folder = ""
raw = folder + "raw_data/"
x_train = pd.read_csv('x_train_preprocess.csv')
x_test = pd.read_csv('x_test_preprocess.csv')
y_train = pd.read_csv(raw + 'y_train.csv', index_col=['id'])
x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

In [41]:
x_train = x_train.iloc[:, 2:]
x_test = x_test.iloc[:, 2:]

In [52]:
x_train

array([[0.38494687, 0.2420187 , 0.73903863, ..., 0.9101033 , 0.98013722,
        0.98074335],
       [0.39479149, 0.08917973, 0.42793298, ..., 0.86740187, 0.85263937,
        0.91733981],
       [0.34401795, 0.05138173, 0.35300931, ..., 0.95804338, 0.97624932,
        0.97617945],
       ...,
       [0.39803438, 0.20228645, 0.44776858, ..., 0.96668633, 0.99328599,
        0.99451794],
       [0.36522961, 0.03241598, 0.3716272 , ..., 0.96058905, 0.98618785,
        0.98555683],
       [0.52549369, 0.13534795, 0.55902059, ..., 0.99974725, 0.99967676,
        0.99933093]])

In [53]:
x_test

array([[0.36242477, 0.1031689 , 0.37398854, ..., 0.92663359, 0.98196061,
        0.96766057],
       [0.28950314, 0.05327929, 0.30292638, ..., 0.98871505, 0.99695465,
        0.99940802],
       [0.42409309, 0.53046124, 0.651658  , ..., 0.9057302 , 0.86012325,
        0.87265384],
       ...,
       [0.40965903, 0.11978037, 0.4698377 , ..., 0.984504  , 0.99781779,
        0.99751903],
       [0.42398821, 0.02214671, 0.42187706, ..., 0.98725353, 0.9863715 ,
        0.98872767],
       [0.54811644, 0.31413584, 0.66781898, ..., 0.99976521, 0.99989535,
        0.99879871]])

In [15]:
from sklearn.ensemble import IsolationForest
clf = IsolationForest(n_estimators=400, random_state=1733979241
)
clf.fit(x_train)
y_pred_train = clf.predict(x_train)
scores = clf.decision_function(x_train)
#count anomalies
anomalies = 0
for i in y_pred_train:
    if i == -1:
        anomalies += 1
print(anomalies)
#remove anomalies
#x_train = x_train[y_pred_train == 1]
#y_train = y_train[y_pred_train == 1]  

183


In [5]:
selector = SelectKBest(score_func=f_classif, k=300)
selector.fit(np.array(x_train), np.array(y_train).squeeze())
cols = selector.get_support(indices=True)

C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [73] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)


In [6]:
#x_train= x_train.iloc[:,cols]
#x_test = x_test.iloc[:,cols]

In [7]:
#feature reduction with pca
from sklearn.decomposition import PCA
pca = PCA(n_components=200)
pca.fit(x_train)
#x_train = pca.transform(x_train)
#x_test = pca.transform(x_test)

PCA(n_components=200)

In [51]:
#scale data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [55]:
import xgboost as xgb
from sklearn.metrics import f1_score
clf = xgb.XGBClassifier(n_estimators=300, objective="multi:softmax", random_state=1733979241, eval_metric=f1_score)

In [37]:
#save the model with pickle
import pickle
filename = 'models/xgboost_model.sav'
pickle.dump(clf, open(filename, 'wb'))


In [ ]:
stop;

In [ ]:
#cross validation score
scores = cross_val_score(clf, x_train, np.ravel(y_train), cv=10, scoring='f1_micro', n_jobs=8)
print(scores.mean())
print(scores.std())
#0.6994274400684931 StandardScaler
#0.6996231347847358 / 0.01592645870348861  MinMaxScaler
#0.6818420529598825 PCA 100
#0.6794921875 PCA 100
#0.7168178968933464 / 0.01259941801705392 new preprocessing
#0.7381195725293541 / 0.014691595901730966 378 features
#0.8133664842221136 / 0.0056350137026039354

In [56]:
clf.fit(x_train, np.ravel(y_train))
y_test = clf.predict(x_test)
data_y_test = {
    'id': [i for i in range(len(y_test))],
    'y': y_test
}
df_y_test = pd.DataFrame(data_y_test)
df_y_test.to_csv('outputs/output_xgb.csv', index=False)

In [ ]:
#feature importance
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
# Print the feature ranking
print("Feature ranking:")
for f in range(x_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))


In [ ]:
#obtain validation set
x_train_2, x_val, y_train_2, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
clf.fit(x_train_2, np.ravel(y_train_2))
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn import metrics
y_pred = clf.predict(x_val)
cm = metrics.confusion_matrix(y_val, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()
plt.show()